In [1]:
import collections
import string
import copy

import functools

In [2]:
def get_grid(filename):
    grid = dict()
    keys = dict()
    doors = dict()
    start = None
    with open(filename) as file:
        for row, line in enumerate(file):
            for col, char in enumerate(line.strip()):
                grid[row, col] = char
                if char == '@':
                    start = (row, col)
                if char in string.ascii_lowercase + '@':
                    keys[char] = (row, col)
                elif char in string.ascii_uppercase:
                    doors[char] = (row, col)

    return grid, keys, doors, start

In [3]:
def draw_grid(grid):
    max_x = max((k[0] for k in grid))
    min_x = min((k[0] for k in grid))
    max_y = max((k[1] for k in grid))
    min_y = min((k[1] for k in grid))

    for x in range(min_x, max_x + 1):
        row = ''.join(grid.get((x, y)) for y in range(min_y, max_y + 1))
        print(row)

In [4]:
def find_neighbours(pos):
    neighbours = []
    for candidate in [
        (pos[0], pos[1] + 1),
        (pos[0], pos[1] - 1),
        (pos[0] + 1, pos[1]),
        (pos[0] - 1, pos[1])]:
        if grid.get(candidate) and not grid.get(candidate) == '#':
            neighbours.append(candidate)
    return neighbours

In [5]:
def find_distances(start):
    visited = set()
    dist = dict()
    dist[start] = 0
    distances = dict()
    queue = collections.deque([(start, '')])

    while queue:
        pos, keys_needed = queue.pop()
        visited.add(pos)
        char = grid[pos]
        
        # We found a key!
        if char in keys and pos != start:
            distances[char] = dist[pos], keys_needed
            keys_needed += char.lower()
            
        # Need a key to pass this door
        if char in doors:
            keys_needed += char.lower()

        # Enqueue the unvisited neighbours of this point
        for n in find_neighbours(pos):
            if n not in visited:
                dist[n] = dist[pos] + 1
                queue.appendleft((n, keys_needed))

    return distances

In [6]:
def add_edges(path):
    pos = keys[path[-1]]
    options = find_reachable(path)

    # print("Started at {}, options {}".format(path, options))
    
    if not options:
        return
    
    for to_key, to_dist in options:
        if to_key in path:
            continue
        new_path = path + to_key
        graph[path].append(new_path)
        dist[new_path] = dist[path] + to_dist
        
        # Call ourself again from this new path
        add_edges(new_path)

In [7]:
def find_reachable(path):
    reachable = []
    from_key = path[-1]
    for dest, (dist, needed) in distances[from_key].items():
        if dest in path:
            continue
        if not (set(needed) - set(path)):
            reachable.append((dest, dist))
    return reachable

In [17]:
grid, keys, doors, start = get_grid("day18.input")

# Compute all distances with keys needed to reach each key
distances = dict()
for from_key in keys:
    distances[from_key] = find_distances(keys[from_key])

draw_grid(grid)

#################################################################################
#.....#...............#.....#.A.#.......#.....#e..............#.....#...........#
#.###.#########.#####.###.#.###.#.###.#.###.#.###.###########.#.#.###.#####.#####
#.#.#.....#...#.#...#.#...#.....#.#.#.#.#...#..m#...#.......#.#.#.#...#.#...#...#
#.#.#####.#.#.#.#.###.#.#########.#.#.###.#####.###.#.#######.###.#.###.#N###.#.#
#.#.#...#...#...#...#.#.....T.......#...#.#...#.....#.#.....#.#...#.....#.#...#.#
#.#.#.#.###########.#.#.###############.#.#.#.#######.#.###.#.#.#.#####.#.#.###.#
#.#...#.......#.....#.#.......#.....#...#.#.#...#.........#j....#.#...#.#.#.#...#
#.###########.#.#.#.#.#########.###.#.#.#.#.###.#.#########.#####.###.#.#.#.#.#.#
#.#.....#...#.#.#.#.#...........#...#.#.#.#...#.#.#...#...#.#...#...#.#.#...#.#.#
#.#P###.#D#.#.#.#.###############.###.#.#.###.#.#.###.#.#.###.#.###.#.#.#####.###
#...#...#.#.#u..#.......#...#.....#...#.#.#...#.......#.#.....#...#...#.....#...#
#####.###.#.####

In [18]:
graph = collections.defaultdict(list)
dist = {'@': 0}

add_edges('@')

KeyboardInterrupt: 

In [16]:
best_route = min(dist, key=lambda x: dist[x] if len(x) == len(keys) else float('inf'))
print(best_route, dist[best_route])

@bacdfeg 132


In [11]:
graph

defaultdict(list,
            {'@': ['@a'],
             '@a': ['@ab'],
             '@ab': ['@abc'],
             '@abc': ['@abce', '@abcd'],
             '@abce': ['@abced'],
             '@abced': ['@abcedf'],
             '@abcd': ['@abcde'],
             '@abcde': ['@abcdef']})

In [12]:
dist

{'@': 0,
 '@a': 2,
 '@ab': 8,
 '@abc': 18,
 '@abce': 32,
 '@abced': 70,
 '@abcedf': 114,
 '@abcd': 42,
 '@abcde': 80,
 '@abcdef': 86}